In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

AUDIO_FOLDER = os.path.abspath("audio_files")
AUDIO_FOLDER_CONVERTED = os.path.abspath("audio_files_converted")
OUTPUT_TRANSCRIPTIONS = os.path.abspath("trascrizioni")

def transcribe_audio(file_path: str) -> str:
   
    with open(file_path, "rb") as audio_file:
        resp = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text"
        )
    return resp

def save_transcription(text: str, output_path: str) -> None:

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)

In [1]:
import os
import whisper

AUDIO_FOLDER = os.path.abspath("audio_files")
AUDIO_FOLDER_CONVERTED = os.path.abspath("audio_files_converted")
OUTPUT_TRANSCRIPTIONS = os.path.abspath("trascrizioni")

model = whisper.load_model("medium")

def transcribe_audio(file_path: str) -> str:
    result = model.transcribe(file_path)
    return result["text"]

def save_transcription(text: str, output_path: str) -> None:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)

if __name__ == "__main__":
    for filename in os.listdir(AUDIO_FOLDER):
        full_path = os.path.join(AUDIO_FOLDER, filename)
        if os.path.isfile(full_path):
            print(f"Trascrivo: {filename}…")
            text = transcribe_audio(full_path)
            out_file = os.path.join(OUTPUT_TRANSCRIPTIONS, os.path.splitext(filename)[0] + ".txt")
            save_transcription(text, out_file)
            print(f"  → Salvato in: {out_file}")

In [2]:
for filename in os.listdir(AUDIO_FOLDER_CONVERTED):
        full_path = os.path.join(AUDIO_FOLDER_CONVERTED, filename)
        if os.path.isfile(full_path):
            print(f"Trascrivo: {filename}…")
            text = transcribe_audio(full_path)
            out_file = os.path.join(OUTPUT_TRANSCRIPTIONS, os.path.splitext(filename)[0] + ".txt")
            save_transcription(text, out_file)
            print(f"  → Salvato in: {out_file}")

Trascrivo: WhatsApp Audio 2025-05-29 at 18.54.34.mp4…


/Users/domenicodemarchis/Desktop/IAARTS/openAI_Whisper_Transcription/.venv/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


  → Salvato in: /Users/domenicodemarchis/Desktop/IAARTS/openAI_Whisper_Transcription/trascrizioni/WhatsApp Audio 2025-05-29 at 18.54.34.txt


In [2]:
from pydub import AudioSegment

def convert_ogg(input_path, output_path, output_format):
    
    if not os.path.isfile(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")
    if not input_path.lower().endswith('.ogg'):
        raise Exception("Il File dovrebbe finire con .ogg") 
    
    base, _ = os.path.splitext(os.path.basename(input_path))
    output_path = os.path.join(output_path, f"{base}.{output_format}")
        
    audio = AudioSegment.from_ogg(input_path)
    audio.export(output_path, format=output_format)

    return output_path


def split_audio_in_two(input_path: str, output_dir: str):

    size_mb = os.path.getsize(input_path) / (1024 * 1024)
    if size_mb <= 25:
        raise ValueError(f"Il file è di soli {size_mb:.2f} MB; niente da splittare.")

    audio = AudioSegment.from_file(input_path)
    duration_ms = len(audio)

    midpoint = duration_ms // 2

    first_half = audio[:midpoint]
    second_half = audio[midpoint:]

    os.makedirs(output_dir, exist_ok=True)

    base_name, ext = os.path.splitext(os.path.basename(input_path))
    ext = ext.replace(".", "")  # es. "mp3", "wav", ecc.

    out1 = os.path.join(output_dir, f"{base_name}_part1.{ext}")
    out2 = os.path.join(output_dir, f"{base_name}_part2.{ext}")
    first_half.export(out1, format=ext)
    second_half.export(out2, format=ext)

    print(f"Splitting completato:\n - {out1}\n - {out2}")

In [7]:
audio_files = [f for f in os.listdir(AUDIO_FOLDER)]

for fname in audio_files:
    base, ext = os.path.splitext(os.path.basename(fname))
    path_audio = os.path.join(AUDIO_FOLDER,fname)
    if ext == '.ogg': path_audio = convert_ogg(fname, AUDIO_FOLDER_CONVERTED, '.mp3')
    size_mb = os.path.getsize(path_audio) / (1024 * 1024)
    if size_mb > 25: split_audio_in_two(path_audio, AUDIO_FOLDER_CONVERTED)

In [8]:
audio_files_converted = [f for f in os.listdir(AUDIO_FOLDER_CONVERTED)]

for fname in audio_files_converted:
    audio_file = os.path.join(AUDIO_FOLDER_CONVERTED, fname)
    base, ext = os.path.splitext(os.path.basename(fname))
    try:
        text = transcribe_audio(audio_file)
        print("=== Trascrizione Eseguita ===")
        save_transcription(text, os.path.join(OUTPUT_TRANSCRIPTIONS,fname))
    except Exception as e:
        print(f"Errore durante la trascrizione: {e}")
        exit(1)

=== Trascrizione Eseguita ===
